In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings('default') #Turns warninigs back on
#Creating a connection
server = 'PLMSSQL64'
database = 'BI_DM_Asset_Protection'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

In [ ]:
#use the connection created above, now using a query to pull data we need

#Enter the company initials
CompanyInitials = 'ATL'
#for multiple companies
#CompanyInitials= 'GRL\',\'GLP'
#query that grabs data
query1='''
SELECT cast(effectivemileage as int) effmiles, cast(monthsinforce as int) as MIF,cast(remainingmonths as int) as RM,--*,

c.agreementid,c.agreementstatus,c.CompanyCode,c.GiveAwayFlag,--c.ProductID, 
ProducerName, Producerstate, c.EffectiveDate,c.ExpirationDate,c.TerminationDate,c.CancelEffectiveDate,cm.WarrantyCompanyReserve,cm.filedamount,--cm.chargedamount,
cm.netmanagementfee,cm.dealercost,
case when c.CancelPercent = 0 then cm.WarrantyCompanyReserve
else ((1-c.CancelPercent)*cm.WarrantyCompanyReserve) 
end as NetWrittenPremium,
cm.EarnedReserve,--c.CancelPercent,
case when c.UsedFactoryWarrantyCoverage = 'Y' then 'Extended'
else c.NewUsedExtended
end as NUE,
c.VSCCertifiedFlag,c.UsedFactoryWarrantyCoverage,--c.newusedextended,
p.ProductGroupCode,p.ProductCategory,p.ProductDescription,p.productcode,
v.VehicleMake,v.VehicleModel,v.vehicleyear,--cm.ParticipationType,
z.paidloss,z2.R12PL,z3.R3PL,--pr.Program,
z.claimct,
1 as ct,
producercode,

year(saledate) as sale_yr,
month(saledate) as sale_mn,
concat(year(saledate),RIGHT('0' + RTRIM(MONTH(saledate)), 2)) as sale_YYYYMM,
concat(termmonths,'/',Termmiles) as Term,
termmonths,termmiles,



CASE WHEN CompanyCode not in ('USW', 'APD', 'PAS','Nww') then 'DOWC'
	WHEN j.PartType IS NULL AND dl.ProgramDirectRetroWarehoused = 'W' THEN 'Warehouse' 
	WHEN j.PartType IS NULL AND dl.ProgramDirectRetroWarehoused = 'C' THEN 'Ceded' 
	WHEN j.PartType IS NULL AND dl.ProgramDirectRetroWarehoused = 'D' THEN 'Direct' 
	WHEN j.PartType IS NULL AND dl.ProgramDirectRetroWarehoused = 'R' THEN 'Retro' -- j.PartType IS NULL AND
	ELSE ltrim(rtrim(j.Description)) END AS PartType,


case when cast(effectivemileage as int) <= 40000 then '000-040'
	 when cast(effectivemileage as int) <= 60000 then '040-060'
	 when cast(effectivemileage as int) <= 80000 then '060-080'
	 when cast(effectivemileage as int) <= 100000 then '080-100'
	 when cast(effectivemileage as int) <= 120000 then '100-120'
	 when cast(effectivemileage as int) <= 130000 then '120-130'
	 when cast(effectivemileage as int) <= 140000 then '130-140'
else '140+   '
end as effmilesband,
case when vehiclemake in ('ACUR','DAEW','GENE','HOND','HYUN','INFI','ISUZ','KIA','LEXU','MAZD','MITS','NISS','SUBA','TOYO') then 'Asian'
	 when vehiclemake in ('BUIC','CADI','CHEV','CHRY','DODG','EAGL','FORD','GEO','GMC','HUMM','JEEP','LINC','MERC','OLDS','PLYM','PONT','RAM','SATU','VOLK') then 'Domestic/VW'
	 when vehiclemake in ('ALFA','AUDI','BENZ','BMW','FIAT','JAGU','LAMO','LAND','MINI','PORS','SAAB','SMAR','SUZU','VOLV','MASE') then 'European'
else ''
end as ADE,
case when year(saledate)-vehicleyear < 0 then 0
	else year(saledate)-vehicleyear
end as MYA,
case when c.ContractEntryDate > eomonth(dateadd(month,-13,eomonth(getdate()))) then 1
else 0 
end as last12,

cm.EarnedReserve-coalesce(ccv.EarnedReserve,0) as R12ER,
cm.EarnedReserve-coalesce(ccv2.EarnedReserve,0) as R3ER--,

from [BI_DM_Asset_Protection].[dbo].[ContractCoverage] as c				
	join [BI_DM_Asset_Protection].[dbo].[Product] as p			
		on c.ProductID = p.ProductID		
	join[BI_DM_Asset_Protection]. [dbo].[Company] as co			
		on co.CompanyInitials = c.CompanyEnteredInitials		
	join [BI_DM_Asset_Protection].[dbo].[Producer] as pr			
		on c.ProducerID = pr.ProducerID		
	inner join
	[BI_DM_Asset_Protection].[dbo].[stage_USW_Alldlr] as dl
	--(select distinct Companyownedinitials,dealercode, Dealergroupingcodeforcommissionreport,ProgramDirectRetroWarehoused,Idcolumn
	--from [BI_DM_Asset_Protection].[dbo].[stage_USW_Alldlr]
	--where Dealergroupingcodeforcommissionreport not in ('')
	--) as  dl
	on pr.producercode=dl.dealercode and pr.ProducerInitials=dl.Companyownedinitials
	left outer join [BI_DM_Asset_Protection].[dbo].[stage_USW_DealerPlanParticipation] AS j ON 
	j.IdColDlr = dl.Idcolumn and
	left(p.ProductGroupCode,1)= j.PlnTyp COLLATE SQL_Latin1_General_CP1_CS_AS
	join [BI_DM_Asset_Protection].[dbo].[ContractMeasures] as cm			
		on c.AgreementID = cm.AgreementID 		
			and c.ProductID = cm.ProductID	
		left join
		(SELECT  aa.agreementid,sum(bb.partcost) as PartTotal,sum(bb.labortotal) as LaborTotal, sum(bb.Laborhours) as laborhours
 FROM [BI_DM_Asset_Protection].[dbo].[ClaimMeasures] aa
	left join [BI_DM_Asset_Protection].[dbo].[ClaimDetail] bb
	on aa.claimid=bb.claimid
	--where aa.claimid='3872380'
	group by aa.AgreementID) as cc
	on c.AgreementID=cc.AgreementID
	
	join [BI_DM_Asset_Protection].[dbo].[Vehicle] v			
	on c.VehicleID=v.VehicleID			
	left outer join 			
	(select distinct Companyinitials, Companytirestickersyn			
	from [BI_DM_Asset_Protection].[dbo].[stage_USW_AllSys]			
	--where Companytirestickersyn='A'			
	) as st			
	on co.CompanyInitials=st.Companyinitials			
		left outer join		
		(select distinct agreementid,productid,count(agreementid) as claimct,sum(IncurredClaimAmount) as incurredloss, sum(paidclaimamount) as paidloss		
	from [BI_DM_Asset_Protection].[dbo].[ClaimMeasures]			
	--where ReportingPeriod < '20200201'			
	group by AgreementID, ProductID) as z
	on c.AgreementID=z.AgreementID and c.ProductID=z.productid	
		left outer join		
		(select distinct agreementid,productid,count(agreementid) as claimct12,sum(IncurredClaimAmount) as incurredloss12, sum(paidclaimamount) as R12PL		
	from [BI_DM_Asset_Protection].[dbo].[ClaimMeasures]			
	where ReportingPeriod >  eomonth(dateadd(month,-13,eomonth(getdate())))			
	group by AgreementID, ProductID) as z2			
	on c.AgreementID=z2.AgreementID and c.ProductID=z2.productid
		left outer join		
		(select distinct agreementid,productid,count(agreementid) as claimct3,sum(IncurredClaimAmount) as incurredloss3, sum(paidclaimamount) as R3PL		
	from [BI_DM_Asset_Protection].[dbo].[ClaimMeasures]			
	where ReportingPeriod >  eomonth(dateadd(month,-4,eomonth(getdate())))			
	group by AgreementID, ProductID) as z3			
	on c.AgreementID=z3.AgreementID and c.ProductID=z3.productid	
	left join [BI_DM_Asset_Protection].[dbo].[ContractCoverageValuations] as ccv
	on c.agreementid=ccv.agreementid and c.productid=ccv.productid and ccv.ValuationDate=dateadd(month,-13,dateadd(month, datediff(month,0,getdate()),0))
	left join [BI_DM_Asset_Protection].[dbo].[ContractCoverageValuations] as ccv2
	on c.agreementid=ccv2.agreementid and c.productid=ccv2.productid and ccv2.ValuationDate=dateadd(month,-4,dateadd(month, datediff(month,0,getdate()),0))

  where --producercode in ('95357','95358') --and c.agreementid in ('86639440','86638185')--and producercode <= '72314'
 
--and  
CompanyCode in (\''''+CompanyInitials+'''\') --and VehicleMake in ('HOND','HYUN','MAZD') 
--and c.EffectiveDate<>c.CancelEffectiveDate
and c.effectivedate >= '2001-01-01 00:00:00.000'
--and ProductCategory='Warranty' --and (ProductDescription like'BASIC%' or ProductDescription like'VALUE%' or ProductDescription like'Ultimate%')
and AgreementStatus in ('Active','Cancelled','Expired') --and c.AgreementID='102408164'
and (p.PL634Flag='Y' or  p.productgroupcode = 'n'     or   p.productgroupcode = 'D')

--and pr.program in ('direct','Warehoused')
--and cm.ParticipationType is not null
   --and ProductGroupCode='K'
   '''
query2='''
-- drop table #A

; with ccvPrior as (
	select 
		Valuationdate,
		year(valuationdate) valuationyear,
		case when month(valuationdate)>=10 then cast(month(valuationdate) as nchar) else CONCAT('0',cast(month(valuationdate) as nchar)) end valuationmonth,
		ccv.agreementid,
		ccv.productid,
		1 as ContractCount,
	    isnull(PaidClaimCount,0) as PaidClaimCount,
		isnull(PaidClaims,0) as PaidClaims,
		isnull(EarnedReserve,0) as EarnedReserve,
		isnull(warrantycompanyreserve,0) as warrantycompanyreserve,
		case when cc.CancelPercent = 0 then ccv.WarrantyCompanyReserve
		else ((1-cc.CancelPercent)*ccv.WarrantyCompanyReserve) 
		end as NetWrittenPremium
		
	from [BI_DM_Asset_Protection].[dbo].[ContractCoverageValuations] ccv join ContractCoverage as CC on ccv.AgreementID=cc.AgreementID and ccv.ProductID=cc.ProductID
	
)
, ccv as (
select
*,
case when isnull(NetWrittenPremium,0)=0 then 0 else isnull(EarnedReserve/NetWrittenPremium,0) end EarnedExposureEst
from ccvPrior

)

,dat as (
select
ccv.ValuationDate,
concat(valuationyear,valuationmonth) valuationmonth,
ccv.AgreementID,
ccv.ProductID,
cc.AgreementStatus,

ccv.ContractCount ContractCountITD,
ccv.PaidClaimCount PaidClaimCountITD,
ccv.PaidClaims PaidClaimsITD,
ccv.EarnedReserve EarnedReserveITD,
ccv.EarnedExposureEst EarnedExposureEstITD,
ccv.warrantycompanyreserve warrantycompanyreserveITD,

ccv.ContractCount - lag(ccv.ContractCount,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) ContractCountMonthly,
ccv.PaidClaimCount - lag(ccv.PaidClaimCount,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) PaidClaimCountMonthly,
ccv.PaidClaims - lag(ccv.PaidClaims,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) PaidClaimsMonthly,
ccv.EarnedReserve - lag(ccv.EarnedReserve,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) EarnedReserveMonthly,
ccv.EarnedExposureEst - lag(ccv.EarnedExposureEst,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) EarnedExposureEstMonthly,
ccv.warrantycompanyreserve - lag(ccv.warrantycompanyreserve,1,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) warrantycompanyreserveMonthly,

ccv.ContractCount - lag(ccv.ContractCount,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) ContractCountR12,
ccv.PaidClaimCount - lag(ccv.PaidClaimCount,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) PaidClaimCountR12,
ccv.PaidClaims - lag(ccv.PaidClaims,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) PaidClaimsR12,
ccv.EarnedReserve - lag(ccv.EarnedReserve,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) EarnedReserveR12,
ccv.EarnedExposureEst - lag(ccv.EarnedExposureEst,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) EarnedExposureEstR12,
ccv.warrantycompanyreserve - lag(ccv.warrantycompanyreserve,12,0) over(partition by ccv.AgreementID, ccv.ProductID order by valuationdate) warrantycompanyreserveR12,
	

p.ProductGroupCode,
rtrim(p.ProductGroupDescription) ProductGroupDescription,
rtrim(p.ProductDescription) ProductDescription,
rtrim(p.ProductCategory) ProductCategory,
year(saledate) as sale_yr,
pr.ProducerCode,
pr.ProducerName,
cc.[CompanyEnteredInitials] ,
co.[CompanyName] as CompanyEnteredInitialsName,
case when cc.UsedFactoryWarrantyCoverage = 'Y' then 'Extended'
else cc.NewUsedExtended
end as NUE,
cc.[CompanyCode],
cc.[CompanyName],
ProducerState,
VehicleMake,
pr.program,
case when vehiclemake in ('ACUR','DAEW','GENE','HOND','HYUN','INFI','ISUZ','KIA','LEXU','MAZD','MITS','NISS','SUBA','TOYO') then 'Asian'
                when vehiclemake in ('BUIC','CADI','CHEV','CHRY','DODG','EAGL','FORD','GEO','GMC','HUMM','JEEP','LINC','MERC','OLDS','PLYM','PONT','RAM','SATU','VOLK','TESL') then 'Domestic/VW'
                when vehiclemake in ('ALFA','AUDI','BENZ','BMW','FIAT','JAGU','LAMO','LAND','MINI','PORS','SAAB','SMAR','SUZU','VOLV','MASE') then 'European'
else ''
end as ADE



from ccv inner join  Product as p on ccv.ProductID=p.ProductID 
inner join [BI_DM_Asset_Protection].[dbo].[ContractCoverage] cc on ccv.AgreementID= cc.AgreementID and ccv.ProductID=cc.ProductID
join[BI_DM_Asset_Protection]. [dbo].[Company] as co
on co.CompanyInitials = cc.CompanyEnteredInitials
join [BI_DM_Asset_Protection].[dbo].[Producer] as pr --this excludes some rows
on cc.ProducerID = pr.ProducerID
left join
(select distinct Companyownedinitials,dealercode, Dealergroupingcodeforcommissionreport,ProgramDirectRetroWarehoused
from [BI_DM_Asset_Protection].[dbo].[stage_USW_Alldlr]
where Dealergroupingcodeforcommissionreport not in ('')
) as dl
on pr.producercode=dl.dealercode and pr.ProducerInitials=dl.Companyownedinitials
join [BI_DM_Asset_Protection].[dbo].[ContractMeasures] as cm
on cc.AgreementID = cm.AgreementID
and cc.ProductID = cm.ProductID
join [BI_DM_Asset_Protection].[dbo].[Vehicle] v -- this excludes further
on cc.VehicleID=v.VehicleID

--inner join Producer pr  on pr.ProducerID=cc.ProducerID
--join [BI_DM_Asset_Protection].[dbo].[Company] as co on co.CompanyInitials = cc.CompanyEnteredInitials


where (p.PL634Flag='Y' or  p.productgroupcode = 'n' or p.productgroupcode = 'D') --and producercode in ('95357','95358')
--and ProductCategory='Warranty'
and cc.CompanyEnteredInitials in (\''''+CompanyInitials+'''\')
--and cc.CompanyName like 'Harbor Assurance%'
--order by 1,2,3
)
--drop table #A
--Select * into #A from dat

select
--ValuationDate,
valuationmonth,
sale_yr,
---AgreementID,
--ProductID,
AgreementStatus,
ProducerCode,
ProducerName,
ProducerState,
[CompanyEnteredInitials] ,
[CompanyName],
ProductGroupDescription,
ProductDescription,
NUE,
VehicleMake,
ADE,
Program,


sum(ContractCountITD) ContractCountITD,
sum(PaidClaimCountITD) PaidClaimCountITD,
sum(PaidClaimsITD) PaidClaimsITD,
sum(EarnedReserveITD) EarnedReserveITD,
sum(EarnedExposureEstITD) EarnedExposureEstITD,
sum(WarrantycompanyreserveITD) WarrantycompanyreserveITD,

sum(ContractCountMonthly) ContractCountMonthly,
sum(PaidClaimCountMonthly) PaidClaimCountMonthly,
sum(PaidClaimsMonthly) PaidClaimsMonthly,
sum(EarnedReserveMonthly) EarnedReserveMonthly,
sum(EarnedExposureEstMonthly) EarnedExposureEstMonthly,
sum(WarrantycompanyreserveMonthly) WarrantycompanyreserveMonthly,

sum(ContractCountR12) ContractCountR12,
sum(PaidClaimCountR12) PaidClaimCountR12,
sum(PaidClaimsR12) PaidClaimsR12,
sum(EarnedReserveR12) EarnedReserveR12,
sum(EarnedExposureEstR12) EarnedExposureEstR12,
sum(WarrantycompanyreserveR12) WarrantycompanyreserveR12,
--ProductGroupCode,

ProductCategory
--year(saledate) as sale_yr
from dat
group by 

valuationmonth,
sale_yr,
---AgreementID,
--ProductID,
AgreementStatus,
ProducerCode,
ProducerName,
ProducerState,
[CompanyEnteredInitials] ,
[CompanyName],
ProductGroupDescription,
ProductDescription,
NUE,
VehicleMake,
ADE,
Program,
ProductCategory
'''

dfall=pd.read_sql(query1,cnxn)

dfall2=pd.read_sql(query2,cnxn)

print('finished')

In [ ]:
print(dfall)
df=dfall[dfall['ProductCategory'] == 'Warranty']
dfan=dfall[dfall['ProductCategory'] == 'Ancillary']
df2=dfall2[dfall2['ProductCategory'] == 'Warranty']
df2an=dfall2[dfall2['ProductCategory'] == 'Ancillary']

In [ ]:
#Creating the rolling 12 dataframes for LR, Freq, and Sev
def calculate_loss_ratio(df, filter_col=None, filter_val=None):
    if filter_col and filter_val:
        df = df[df[filter_col] == filter_val]
    df = df.groupby(['valuationmonth']).apply(lambda x: pd.Series({'LossRatioR12':(x['PaidClaimsR12'].sum()/x['EarnedReserveR12'].sum()).round(3)}))
    if not df.empty:
        df = df.tail(16).reset_index()
        df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
    return df

def calculate_frequency(df, filter_col=None, filter_val=None):
    if filter_col and filter_val:
        df = df[df[filter_col] == filter_val]
    df = df.groupby(['valuationmonth']).apply(lambda x: pd.Series({'FrequencyR12':(x['PaidClaimCountR12'].sum()/(x['ContractCountR12'].sum()*(x['EarnedReserveR12'].sum()/x['WarrantycompanyreserveR12'].sum()))).round(3)}))
    if not df.empty:
        df = df.tail(16).reset_index()
        df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
    return df

def calculate_severity(df, filter_col=None, filter_val=None):
    if filter_col and filter_val:
        df = df[df[filter_col] == filter_val]
    df = df.groupby(['valuationmonth']).apply(lambda x: pd.Series({'SeverityR12':(x['PaidClaimsR12'].sum()/x['PaidClaimCountR12'].sum()).round(0)}))
    if not df.empty:
        df = df.tail(16).reset_index()
        df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
    return df

# Create the dataframes LR
dfR12LR = calculate_loss_ratio(df2)
dfR12LRn = calculate_loss_ratio(df2, 'NUE', 'New')
dfR12LRu = calculate_loss_ratio(df2, 'NUE', 'Used')
dfR12LRe = calculate_loss_ratio(df2, 'NUE', 'Extended')
dfR12LRaz = calculate_loss_ratio(df2, 'ADE', 'Asian')
dfR12LRdo = calculate_loss_ratio(df2, 'ADE', 'Domestic/VW')
dfR12LReu = calculate_loss_ratio(df2, 'ADE', 'European')
# Freq
dfR12Frq = calculate_frequency(df2)
dfR12Frqn = calculate_frequency(df2, 'NUE', 'New')
dfR12Frqu = calculate_frequency(df2, 'NUE', 'Used')
dfR12Frqe = calculate_frequency(df2, 'NUE', 'Extended')
dfR12Frqaz = calculate_frequency(df2, 'ADE', 'Asian')
dfR12Frqdo = calculate_frequency(df2, 'ADE', 'Domestic/VW')
dfR12Frqeu = calculate_frequency(df2, 'ADE', 'European')
# Sev
dfR12Sev = calculate_severity(df2)
dfR12Sevn = calculate_severity(df2, 'NUE', 'New')
dfR12Sevu = calculate_severity(df2, 'NUE', 'Used')
dfR12Seve = calculate_severity(df2, 'NUE', 'Extended')
dfR12Sevaz = calculate_severity(df2, 'ADE', 'Asian')
dfR12Sevdo = calculate_severity(df2, 'ADE', 'Domestic/VW')
dfR12Seveu = calculate_severity(df2, 'ADE', 'European')
print(dfR12LReu)

In [ ]:
def create_grouped_df(df, groupby_col, filter_col=None, filter_val=None,filter_col2=None,filter_val2=None):
    # Filter the DataFrame based on the specified filter columns and values, if provided
    if filter_col and filter_val:
        df_filtered = df[df[filter_col] == filter_val]
    else:
        df_filtered = df
    if filter_col2 and filter_val2:
        df_filtered = df_filtered[df_filtered[filter_col2] == filter_val2]
    # Group the filtered DataFrame by the specified groupby column and apply the lambda function
    df_grouped = df_filtered.groupby([groupby_col]).apply(lambda x: pd.Series({
        'Contract_Count': x['ct'].sum(),
        'Last12_Contracts': x['last12'].sum(),
        'TotalWrittenPremium': x['WarrantyCompanyReserve'].sum(),
        'NetWrittenPremium': x['NetWrittenPremium'].sum(),
        'EarnedPremium': x['EarnedReserve'].sum(),
        'PaidLosses': x['paidloss'].sum(),
        'Claim_ct': x['claimct'].sum(),
        'LRITD': (x['paidloss'].sum() / x['EarnedReserve'].sum()).round(3),
        'R12LR': (x['R12PL'].sum()/x['R12ER'].sum()).round(3),
        'R3LR': (x['R3PL'].sum()/x['R3ER'].sum()).round(3),
        'Earned %': (x['EarnedReserve'].sum()/x['WarrantyCompanyReserve'].sum() ).round(3),
        'Frequency': (x['claimct'].sum()/(x['ct'].sum()*(x['EarnedReserve'].sum()/x['WarrantyCompanyReserve'].sum()))).round(3),
        'Severity': (x['paidloss'].sum()/x['claimct'].sum()).round(0),
        'AVG_Written_Premium': x['WarrantyCompanyReserve'].sum()/x['ct'].sum(),
        'Pure_Premium': x['paidloss'].sum()/(x['ct'].sum()*(x['EarnedReserve'].sum()/x['WarrantyCompanyReserve'].sum()))
    }))
    # Sort by Contract count highest to lowest
    df_grouped = df_grouped.sort_values(by='Contract_Count', ascending=False)
    # Change contract count to int so we don't get decimals
    df_grouped['Contract_Count'] = df_grouped['Contract_Count'].astype(int)
    
    # Add a total row at the bottom for the filter applied
    total_row = pd.Series(df_grouped.sum(numeric_only=True), name='Total')
    total_row[['LRITD']] = df_filtered[['paidloss', 'EarnedReserve']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(3))
    total_row[['R12LR']] = df_filtered[['R12PL', 'R12ER']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(3))
    total_row[['R3LR']] = df_filtered[['R3PL', 'R3ER']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(3))
    total_row[['Earned %']] = df_filtered[['EarnedReserve', 'WarrantyCompanyReserve']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(3))
    total_row[['Frequency']] = df_filtered[['claimct', 'ct', 'EarnedReserve', 'WarrantyCompanyReserve']].agg('sum').pipe(lambda x: (x[0]/(x[1]*(x[2]/x[3]))).round(3))
    total_row[['Severity']] = df_filtered[['paidloss', 'claimct']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(0))
    total_row[['AVG_Written_Premium']] = df_filtered[['WarrantyCompanyReserve', 'ct']].agg('sum').pipe(lambda x: (x[0]/x[1]).round(0))
    total_row[['Pure_Premium']] = df_filtered[['paidloss','ct','EarnedReserve','WarrantyCompanyReserve']].agg('sum').pipe(lambda x: (x[0]/(x[1]*(x[2]/x[3]))).round(0))
    df_grouped = df_grouped.append(total_row)
    
    return df_grouped
#creating dataframe arguments: df, groupby col, filter col1, filter value1, filter col2, filter value2
dfdealer= create_grouped_df(df, 'ProducerName')
dfNUE= create_grouped_df(df, 'NUE')
dfnp = create_grouped_df(df,'ProductDescription','NUE', 'New')
dfup = create_grouped_df(df,'ProductDescription', 'NUE', 'Used')
dfep = create_grouped_df(df,'ProductDescription', 'NUE', 'Extended')
dfupD=create_grouped_df(df,'ProductDescription','NUE','Used','ADE','Domestic/VW')
print(dfdealer)

# Create a plot with the DataFrame

In [ ]:
df.plot()

# Add a title to the plot

In [ ]:
import matplotlib.pyplot as plt
plt.title("My DataFrame")
plt.show()

In [ ]:
#fill the na values with zero
dfdealer.fillna(0,inplace=True)
dfNUE.fillna(0,inplace=True)
dfnp.fillna(0,inplace=True)
dfup.fillna(0,inplace=True)
dfep.fillna(0,inplace=True)
dfupD.fillna(0,inplace=True)
#had to insert the code above. Helps fill na values to help with calculations below
#another option was to add "options={'nan_inf_to_errors': True}" in the writer function below after engine
#went with fillna option as it looks better with 0's in the excel file.

# Replace multiple consecutive spaces with a single space in the first column
# have to reset the index. First column isn't being picked up currently
dfdealer=dfdealer.reset_index()
dfNUE=dfNUE.reset_index()
dfnp = dfnp.reset_index()
dfep = dfep.reset_index()
dfup = dfup.reset_index()
dfupD=dfupD.reset_index()
# Replace multiple spaces with a single space
dfdealer.iloc[:, 0] = dfdealer.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
dfNUE.iloc[:, 0] = dfNUE.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
dfnp.iloc[:, 0] = dfnp.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
dfep.iloc[:, 0] = dfep.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
dfup.iloc[:, 0] = dfup.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)
dfupD.iloc[:, 0] = dfupD.iloc[:, 0].str.replace(r'\s+', ' ', regex=True)

In [ ]:
#df names: dfdealer, dfNUE, dfnp, dfup, dfep

# Create a writer object
file_path ='Z:\\APD\\Risk Management\\Private\\123files\\EEnyart\\Dealer Reports\\test\\'+CompanyInitials+'_Report.xlsx'
writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

# Define a number format for percentage (2 decimal places)
percent_format = writer.book.add_format({'num_format': '0.0%'})
# Define a number format for percentage (2 decimal places) with bold font
percent_format_bold = writer.book.add_format({'num_format': '0.0%', 'bold': True})
# Define a custom number format for comma-separated values rounded to the nearest whole number
comma_format = writer.book.add_format({'num_format': '#,##0'})
# Define a custom number format for comma-separated values rounded to the nearest whole number
comma_format_bold = writer.book.add_format({'num_format': '#,##0', 'bold':True})
# Define a bold format for last row
bold= writer.book.add_format({'bold':True})
# Specify the columns you want to format as percentages
columns_to_format_as_percentages = ['LRITD', 'R12LR', 'R3LR','Earned %','Frequency']
# Specify the columns you want to format with commas and rounded
columns_to_format_with_commas_and_rounded = ['Contract_Count', 'Last12_Contracts','TotalWrittenPremium','AVG_Written_Premium',\
                                'Severity','Pure_Premium','NetWrittenPremium', 'EarnedPremium', 'PaidLosses', 'Claim_ct']

#writing module to loop through dataframes and apply them to sheets
def write_df_to_excel(df, sheet_name, writer):
    # Write the DataFrame to the specified sheet
    df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the worksheet object
    sheet = writer.sheets[sheet_name]
    # Loop through columns and format cells as percentages for the specified columns
    for col_name in columns_to_format_as_percentages:
        col_num = df.columns.get_loc(col_name)# Get the column index 
        sheet.set_column(col_num, col_num, None, percent_format)
        # Get last row index : have to do -1 and add it back below.throws error otherwise I think from df.at part
        last_row = len(df) - 1
        #Format the last cell as bold and percentage
        sheet.write_number(last_row+1,col_num,df.at[last_row,col_name], percent_format_bold)
        
    # Loop through columns and format cells with commas and rounded for the specified columns
    for col_name in columns_to_format_with_commas_and_rounded:
        col_num = df.columns.get_loc(col_name)  # Get the column index
        sheet.set_column(col_num, col_num, None, comma_format)
        last_row = len(df) - 1
        sheet.write_number(last_row+1,col_num,df.at[last_row,col_name], comma_format_bold)
        
    # Auto-adjust column widths
    for i, col_name in enumerate(df.columns):
        max_col_width = max(df[col_name].astype(str).apply(len).max(), len(col_name))
        sheet.set_column(i, i, max_col_width + 2)    # Add some padding to the column width
    
# Write the DataFrames to different sheets
write_df_to_excel(dfdealer, 'Dealers', writer)
write_df_to_excel(dfNUE, 'NUE', writer)
write_df_to_excel(dfnp, 'New Products', writer)
write_df_to_excel(dfup, 'Used Products', writer)
write_df_to_excel(dfep, 'Extended Products', writer)
write_df_to_excel(dfupD, 'Used Domestic Products', writer)

def create_line_chart(sheet_name, chart_title, x_axis_name, y_axis_name, y_axis_num_format, data_frames,\
                      series_names, chart_position, helper_sheet_name):
    # Get a reference to the existing worksheet
    chart_sheet = writer.book.get_worksheet_by_name(sheet_name)
    
    # Create a chart object
    chart = writer.book.add_chart({'type': 'line'})
    
    # Configure the chart series
    for i, df in enumerate(data_frames):
        df.to_excel(writer, sheet_name=helper_sheet_name, startrow=1, startcol=i*2,index=False)
        # Configure the chart series
        chart.add_series({
            'name':       series_names[i],
            'categories': [helper_sheet_name, 2, i*2, len(df)+1, i*2],
            'values':     [helper_sheet_name, 2, i*2+1, len(df)+1, i*2+1],
        })
    
    # Add chart title and axis labels
    chart.set_title({'name': chart_title})
    chart.set_x_axis({'name': x_axis_name})
    chart.set_y_axis({'name': y_axis_name,'num_format':y_axis_num_format})
    
    # Set the size of the chart
    chart.set_size({'width':600,'height':376})
    
    # Insert the chart into the worksheet
    chart_sheet.insert_chart(chart_position, chart)
    # Get a reference to the helper sheet
    helper_sheet = writer.book.get_worksheet_by_name(helper_sheet_name)

    # Hide the helper sheet
    helper_sheet.hide()

# Get the number of rows in dfdealers
num_rows = dfdealer.shape[0]
# Calculate the row number where you want to insert the chart
chart_row = num_rows + 2

# Arguements (sheetname, graphname, x_axis_name, y_axis_name, y_axis_num_format, dataframes, seriesnames, chart position)
create_line_chart('Dealers', 'NUE R12 LR', 'Valuation Month', 'Loss Ratio','0%',\
                  [dfR12LR,dfR12LRn, dfR12LRu, dfR12LRe], ['Overall','New', 'Used', 'Extended'], f'A{chart_row}','Helper_lrNUE')
create_line_chart('Dealers', 'ADE R12 LR', 'Valuation Month', 'Loss Ratio','0%',\
                  [dfR12LR,dfR12LRaz, dfR12LRdo, dfR12LReu], ['Overall','Asian', 'Domestic', 'European'], f'A{chart_row+19}','Helper_lrADE')
create_line_chart('Dealers', 'NUE R12 Freq', 'Valuation Month', 'Frequency','0%',\
                  [dfR12Frq,dfR12Frqn,dfR12Frqu,dfR12Frqe], ['Overall','New', 'Used', 'Extended'], f'E{chart_row}','helper_frqNUE')
create_line_chart('Dealers', 'ADE R12 Freq', 'Valuation Month', 'Frequency','0%',\
                  [dfR12Frq,dfR12Frqaz,dfR12Frqdo,dfR12Frqeu], ['Overall','Asian', 'Domestic', 'European'], f'E{chart_row+19}','helper_frqADE')
create_line_chart('Dealers', 'R12 Sev', 'Valuation Month', 'Severity','#,##0',\
                  [dfR12Sev,dfR12Sevn,dfR12Sevu,dfR12Seve], ['Overall','New', 'Used', 'Extended'], f'K{chart_row}','helper_sevNUE')
create_line_chart('Dealers', 'R12 Sev', 'Valuation Month', 'Severity','#,##0',\
                  [dfR12Sev,dfR12Sevaz,dfR12Sevdo,dfR12Seveu], ['Overall','Asian', 'Domestic', 'European'], f'K{chart_row+19}','helper_sevADE')

# Save the Excel file
writer.save()
writer.close()
print('Finished')

In [ ]:
print('Finished')